<a href="https://colab.research.google.com/github/atharvac1301/Amazon-Review-Classification-BERT/blob/main/multilingual_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch] datasets evaluate sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl

In [ ]:
from datasets import load_dataset

src_lang = "cs"
tgt_lang = "pl"

dataset_name = "euconst"

cs_pl = load_dataset("Helsinki-NLP/euconst", "cs-pl")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/10029 [00:00<?, ? examples/s]

In [ ]:
cs_pl

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 10029
    })
})

In [ ]:
cs_pl = cs_pl["train"].train_test_split(train_size=5000, test_size=1000)
cs_pl

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 1000
    })
})

In [ ]:
# count = 0
# for pair in cs_pl["train"]:
#   if len(pair["translation"]["cs"]) <= 5 or len(pair["translation"]["pl"]) <= 5:
#     count += 1
# count


# cs_pl = [pair for pair in cs_pl if cs_pl["train"]["translation"]["cs"]]


1915

# Load the Tokenizer

In [ ]:
"""
from transformers import NllbTokenizerFast

checkpoint = "facebook/nllb-200-distilled-600M"

tokenizer = NllbTokenizerFast.from_pretrained(
    "facebook/nllb-200-distilled-600M", src_lang=src_lang, tgt_lang=tgt_lang
)
"""

from transformers import M2M100Config, M2M100ForConditionalGeneration, M2M100Tokenizer

checkpoint = "facebook/m2m100_418M"

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="cs", tgt_lang="pl")



config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

In [ ]:
"""
prefix = "translate Czech to Polish: "
(prefix prompt for T5 Models)
"""


def preprocess_function(examples):
    # inputs = [prefix + example[source_lang] for example in examples["translation"]]
    inputs = [example[src_lang] for example in examples["translation"]]
    targets = [example[tgt_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [ ]:
tokenized = cs_pl.map(preprocess_function, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)


# Evaluate

In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

## Compute and Calculate SacreBLEU score

In [ ]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

# Train

In [ ]:
# !pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 1.7 MB/s eta 0:00:00


In [ ]:
# from peft import LoraConfig, TaskType

# peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM,
#                          inference_mode=False,
#                          r=8,
#                          lora_alpha=32,
#                          lora_dropout=0.1,
#                          target_modules=["q_proj", "v_proj"]
#                       )


In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

# model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)


In [ ]:
# from peft import get_peft_model

# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()


trainable params: 1,179,648 || all params: 485,085,184 || trainable%: 0.24318367967305304


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# !huggingface-cli whoami

achou1301


In [ ]:
# !huggingface-cli model search euconst-cs-pl

usage: huggingface-cli <command> [<args>]
huggingface-cli: error: argument {env,login,whoami,logout,repo,upload,download,lfs-enable-largefiles,lfs-multipart-upload,scan-cache,delete-cache}: invalid choice: 'model' (choose from 'env', 'login', 'whoami', 'logout', 'repo', 'upload', 'download', 'lfs-enable-largefiles', 'lfs-multipart-upload', 'scan-cache', 'delete-cache')


In [ ]:
# import torch
# torch.cuda.empty_cache()

In [ ]:
!pip install safetensors

In [ ]:
from safetensors.torch import load_model, save_model


save_model(model, "model.safetensors")
# Instead of save_file(model.state_dict(), "model.safetensors")

load_model(model, "model.safetensors")
# Instead of model.load_state_dict(load_file("model.safetensors"))



In [ ]:
import torch.optim as optim

training_args = Seq2SeqTrainingArguments(
    output_dir=f"{dataset_name}-{src_lang}-{tgt_lang}",
    # overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    save_safetensors = True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,

)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.730000,1.196561,28.711000,25.869000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.730000,1.196561,28.711000,25.869000
2,0.965800,0.948655,36.229500,26.136000


TrainOutput(global_step=1250, training_loss=1.22033583984375, metrics={'train_runtime': 1295.5261, 'train_samples_per_second': 7.719, 'train_steps_per_second': 0.965, 'total_flos': 1436553324134400.0, 'train_loss': 1.22033583984375, 'epoch': 2.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/euconst-cs-pl


/content/euconst-cs-pl


In [ ]:
trainer.push_to_hub()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/achou1301/euconst-cs-pl/commit/e7128b9436887a27c42017a5321bb569c51e7817', commit_message='End of training', commit_description='', oid='e7128b9436887a27c42017a5321bb569c51e7817', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
print("HEllo World")

HEllo World


In [ ]:
# !pip install wandb

In [ ]:
# wandb login